# 2.5. Bayes-Klassifikator

>## <ins>Table of contents</ins>
>* [**2.5.0. Vorwissen: Der Satz von Bayes**](#2_5_0)
>* [**2.5.1. Das Maximum-Likelihood-Prinzip**](#2_5_1)
>* [**2.5.2. Bayes-Klassifikation und lineare Regression**](#2_5_2)
>* [**2.5.3. Naive Bayes-Klassifikation**](#2_5_3)
>* [**2.5.4. Kontinuierliche Merkmale**](#2_5_4)

## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
import matplotlib.pyplot as plt


## 2.5.0. Vorwissen: Der Satz von Bayes <a name="2_5_0"></a>
---

Wir betrachten ein zweistufiges Zufallsexperiment mit zwei Ereignissen `A`und `B`.

**Gegeben**: `P(B|A)`, die Wahrscheinlichkeit von B unter der Bedingung, dass A eingetreten ist.

![image.png](attachment:c1ff66a3-3c47-4482-af8d-2b8ab0ebc483.png)

**Gesucht**:  $P(A|B)$ ist die Wahrscheinlichkeit von A unter der Bedingung, dass B eingetreten ist.

![image.png](attachment:cceba301-99c9-4d78-9522-248c56285fa1.png)

Nach dem Multiplikationssatz gilt: $$P(A \cap B) = P(B) * P(A|B)$$
Gleichung nach `P(A|B)` auflösen: $$P(A|B) = \frac{ P(A \cap B)}{P(B)}$$

von daher der Satz von Bayes ist: $$P(A|B) = \frac{P(A) * P(B|A)}{P(B)}$$
$$P(A|B) = \frac{P(A) * P(B|A)}{P(A) * P(B|A) + P(\overline{A}) * P(B|\overline{A})}$$


>**Beispiel**
>**Eine Schülerin fährt in 70 % der Schultage mit dem Bus. In 80 % dieser Fälle kommt sie pünktlich zur Schule. Durchschnittlich kommt sie aber nur an 60 % der Schultage pünktlich an.**
>**Heute kommt die Schülerin pünktlich zur Schule. Mit welcher Wahrscheinlichkeit hat sie den Bus benutzt?**


>Für die Ereignisse werden folgende Bezeichnungen gewählt:
>
>$A$: Die Schülerin fährt mit dem Bus.
>
>$B$: Die Schülerin kommt pünktlich an.
>
>Demnach gilt:
>
>$\overline{A}$: Die Schülerin fährt nicht mit dem Bus.
>
>$\overline{B}$: Die Schülerin kommt nicht pünktlich an.
>
>Eine Schülerin fährt zu 70 % mit dem Bus: $P(A) = 0.7$
>
>In 80 % dieser Fälle kommt sie pünktlich. $P(B|A) = 0.8$
>
>Durchschnittlich kommt sie zu 60 % pünktlich: $P(B) = 0.6$
>
>Gesucht ist die Wahrscheinlichkeit für BUS unter der Bedingung PÜNKTLICH: `P(B|A)`
>
>Da P(B|A) gegeben und P(A|B) gesucht ist, lösen wir die Aufgabe mit dem Satz von Bayes:
>
>$$P(A|B) = \frac{P(A) * P(B|A)}{P(B)} = \frac{0.7 * 0.8}{0.6} = 0.93 = 93.3%$$

## 2.5.1. Das Maximum-Likelihood-Prinzip <a name="2_5_1"></a>
---
Das fundamentale Problem des maschinellen Lernens ist ein optimales Modell `h` aus einem Hypothesenraum `H` (z.B. der Menge aller linearen Funktionen im Fall der linearen Regression) zu finden, das die Trainingsdaten `D` am besten erklärt. 
Wir suchen die wahrscheinlichste Hypothese `h`, gegeben die beobachteten Daten `D`, dies bedeutet, dass wir den Wert `P(h | D)` maximieren wollen.

### Bayes-Theorem
Mit dem Bayes-Theorem können wir den Wert `P(h | D)` annähern:
$$(1)$$
$$P(h | D) = \frac{P(D | h)P(h)}{P(D)}$$ 

Hierbei ist:
- `P(h | D)` ist die **a posteriori Wahrscheinlichkeit** der Hypothese.
- `P(h)` die **a priori Wahrscheinlichkeit** der Hypothese h, oft angenommen als Gleichverteilung auf H.
- `P(D)` ist die Wahrscheinlichkeit, dass der Datensatz D beobachtet wurde,
- und `P(D | h)` ist die Wahrscheinlichkeit, D zu beobachten, gegeben dass D von h generiert wurde.

#### - Maximum-A-Posteriori-Hypothese (MAP-Hypothese)
Unser Ziel ist es, eine Hypothese `h*` zu finden, die die die a posteriori Wahrscheinlichkeit `P(h | D)` (1) maximiert:
$$(2)$$
$$h* = \arg\max_{h \in H} P(h | D)$$
In anderen Worten, unter allen Hypothesen in `H` suchen wir diejenige, die die höchste Wahrscheinlichkeit hat, gegeben die beobachteten Daten `D`.
Diese Methode berücksichtigt sowohl die Wahrscheinlichkeit der Daten gegeben die Hypothese `P(D | h)` als auch die a priori Wahrscheinlichkeit der Hypothese `P(h)`.

**Beispiel:**

Angenommen, Sie haben eine Münze und möchten herausfinden, ob sie fair ist (d.h., die Wahrscheinlichkeit für Kopf `P(Kopf)` ist 0,5) oder ob sie eine Präferenz für Kopf oder Zahl hat. Sie werfen die Münze 10 Mal und erhalten 7 Mal Kopf und 3 Mal Zahl. Die MAP-Hypothese würde die a priori Annahme berücksichtigen, dass die meisten Münzen fair sind (d.h., `P(h)` ist hoch für `h=0,5`). Daher könnte die MAP-Hypothese trotz der beobachteten Daten immer noch zu dem Schluss kommen, dass die Münze fair ist.


#### - Maximum-Likelihood-Hypothese (ML-Hypothese)
Wenn P(h) gleichverteilt ist, können wir den Term P(h) streichen und erhalten die Maximum-Likelihood-Hypothese (ML-Hypothese). Die ML-Hypothese ist eine Hypothese `h*`, die die Wahrscheinlichkeit `P(D | h)` maximiert, also die Wahrscheinlichkeit, die Daten `D` zu beobachten, gegeben die Hypothese `h`. Dies wird durch die folgende Formel ausgedrückt:

$$h* = \arg\max_{h \in H} P(D | h)$$

>Im Gegensatz zur MAP-Hypothese berücksichtigt die ML-Hypothese nicht die a priori Wahrscheinlichkeit der Hypothese `P(h)`. Stattdessen basiert sie nur auf der Übereinstimmung der Hypothese mit den beobachteten Daten. Dieser Ansatz ist besonders nützlich, wenn alle Hypothesen als gleich wahrscheinlich angesehen werden (P(h) ist gleichverteilt). Es ist ein weit verbreiteter Ansatz im maschinellen Lernen für das Training von Modellen und die Vorhersage von Ergebnissen.

**Beispiel:**
Im gleichen Szenario mit der Münze würde die ML-Hypothese nur die beobachteten Daten berücksichtigen und die Hypothese wählen, die die Daten am besten erklärt, ohne Rücksicht auf irgendwelche a priori Annahmen. In diesem Fall würde die ML-Hypothese zu dem Schluss kommen, dass `P(Kopf)` 0,7 ist, da dies die Wahrscheinlichkeit maximiert, die beobachteten Daten (7 Kopf und 3 Zahl) zu sehen.


## 2.5.2. Die Bayes-Klassifikation und lineare Regression <a name="2_5_2"></a>
---

Es wird ein Datensatz D mit Paaren von x und y gegeben, und das Ziel ist es, Parameter $\theta^*$ zu finden, die die Summe der quadrierten Differenzen zwischen den vorhergesagten und tatsächlichen y-Werten minimieren.

Das gelernte Modell $h_{\theta^*}$ beschreibt eine Gerade im $R^n$, die den funktionalen Zusammenhang zwischen den Merkmalen x und der Zielvariablen y modelliert. Wir nehmen an, dass der Datensatz D verrauscht ist und die Werte $y(i)$ die Form $y(i) = \hat{y}(i) + \epsilon(i)$ haben, wobei $\hat{y}(i)$ der wahre Wert der Merkmalsausprägung $x(i)$ ist und $\epsilon(i)$ der Fehler. Wir nehmen auch an, dass die Fehler normalverteilt sind.

Die Wahrscheinlichkeitsdichte, den Wert $y(i)$ anstatt $\hat{y}(i)$ zu beobachten, ist durch die Normalverteilung gegeben. Die Wahrscheinlichkeitsdichte, ein Beispiel $d(i) = (x(i), y(i))$ unter der Hypothese $\theta$ zu beobachten, ist ebenfalls durch die Normalverteilung gegeben.

Im Bayes'schen Ansatz wird eine ML-Hypothese $\theta^*$ ausgewählt, die die Wahrscheinlichkeitsdichte maximiert. Anstatt den Likelihood zu maximieren, kann auch der Log-Likelihood maximiert werden, da der natürliche Logarithmus eine monotone Funktion ist.

Gegeben ist ein Datensatz `D` mit Merkmalen `Z1` bis `Zn` und Klassen `Z`. Die Idee der Bayes-Klassifikation ist, dass die Hypothese ausgewählt wird, die angesichts der Daten am wahrscheinlichsten ist. Für einen neuen Datenpunkt `x` wählen wir die Klasse `c` aus `Z`, die aufgrund von `D` am wahrscheinlichsten ist. Der Bayes-Klassifikator `clf_Bayes_D` wird formal definiert als:

$$clf_{Bayes_D}(x) = \arg\max_{c \in Z} P(c | x,D)$$

wobei

$$P(c | x,D) = \frac{|{(x, c) \in D}|}{\sum_{c' \in Z} |{(x, c') \in D}|}$$

Einem neuen Datenpunkt `x` wird die Klasse `c` aus `Z` zugewiesen, die am häufigsten `x` in `D` zugewiesen wird.
>**Beispiell** Wir haben einen Trainingsdatensatz `D_movies2` für ein Filmempfehlungssystem. Die Filme werden anhand der binären Merkmale "Hat der Film einen Oskar gewonnen?" (`Z1`) und "Wurde der Film in Europa gedreht?" (`Z2`) klassifiziert. Die Filme werden als "gut" (`Z=1`) oder "schlecht" (`Z=0`) eingestuft.

In [27]:
import numpy as np
import pandas as pd

# Ihr Trainingsdatensatz D_movies2
data = np.array([
    [0, 1, 1],
    [1, 0, 1],
    [0, 0, 0],
    [0, 1, 1],
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 0],
    [1, 0, 0],
    [1, 0, 1],
    [1, 0, 1]
])

# Erstellen Sie einen DataFrame aus den Daten
D_movies2 = pd.DataFrame(data, columns=['Z1', 'Z2', 'Z'])
print(D_movies2)

   Z1  Z2  Z
0   0   1  1
1   1   0  1
2   0   0  0
3   0   1  1
4   0   1  0
5   0   0  1
6   0   0  0
7   1   0  0
8   1   0  1
9   1   0  1


>Sie möchten nun die Wahrscheinlichkeit berechnen, dass ein neuer Datenpunkt `x* = (0,1)` (ein in Europa gedrehter Film, der keinen Oskar gewonnen hat) der Klasse 0 oder 1 zugeordnet wird.
>Hier ist ein einfacher Python-Code, der das tut:

In [28]:
def c_given_x_bayes(x):
    x_bayes = np.all(D_movies2[['Z1', 'Z2']] == x, axis=1)

    p_class_0 = np.mean(D_movies2[x_bayes]['Z']==0)
    p_class_1 = np.mean(D_movies2[x_bayes]['Z']==1)
    
    #p_class_1 = np.mean(good_films)
    
    print(f"Wahrscheinlichkeit für Klasse 0: {p_class_0}")
    print(f"Wahrscheinlichkeit für Klasse 1: {p_class_1}")
    
x_star = np.array([0, 1])
c_given_x_bayes(x_star)

Wahrscheinlichkeit für Klasse 0: 0.3333333333333333
Wahrscheinlichkeit für Klasse 1: 0.6666666666666666


Es gibt drei Beispiele in $D_{\text{movies2}}$, die mit $x^*$ in der Merkmalsausprägung übereinstimmen (Nr. 0, 3 und 4), zwei davon sind als 1 und eins als 0 klassifiziert. Wir erhalten also

$$
P(c = 0 | x = x^*,D = D_{\text{movies2}}) = \frac{1}{3}
$$

$$
P(c = 1 | x = x^*,D = D_{\text{movies2}}) = \frac{2}{3}
$$

Die Klasse 1 kommt also häufiger unter der Merkmalsausprägung $x^*$ vor und deshalb erhalten wir

$$
\text{clf}_{\text{Bayes}}^{D_{\text{movies2}}}(x^*) = 1
$$

Der Film $x^*$ wird also als "gut"
**Problem -->**
1. wir müssen für jede zu erwartende Merkmalsausprägung $x^∗$ Beispiele im Trainingsdatensatz D haben, um $x^∗$ überhaupt klassifizieren zu können.
   Z.B. Für den Datensatz $D_{\text{{movies2}}}$ können wir für den Datenpunkt $x^{**} = (1,1)$ (d.h., ein in Europa gedrehter Film, der einen Oskar gewonnen hat) keine Klassifikation vorhersagen, da alle Wahrscheinlichkeiten undefiniert sind (d.h., gleich $\frac{0}{0}$). klassifiziert.


In [29]:
c_given_x_bayes(np.array([1, 1]))

Wahrscheinlichkeit für Klasse 0: nan
Wahrscheinlichkeit für Klasse 1: nan


2. Ein weiteres Problem (das auch prinzipiell schon beim KNN-Algorithmus aufgetreten ist) besteht darin, dass zur Berechnung von $\text{{clf}}_{\text{{Bayes}}_D}(x)$ stets der gesamte Datensatz $D$ vorgehalten werden muss. Insbesondere bei großen Datensätzen kann dies signifikant zu einer langen Berechnungszeit beitragen.

**Lösung -->** von der <ins>allgemeinen Bayes-Klassifikation</ins> zur <ins>Naiven Bayes-Klassifikation</ins> wechseln.

## 2.5.3. Klassifikationsalgorithmus: Naive Bayes-Klassifikation <a name="2_5_3"></a>
---

Der Klassifikationsalgorithmus Naive Bayes-Klassifikator wird auf Szenarien angewendet, in denen Merkmale in einem endlichen Merkmalsraum definiert sind.

Es unterscheidet sich von der allgemeinen Bayes-Klassifikation darin, dass, bei der Berechnung der bedingten Wahrscheinlichkeit $P(c | x,D)$, also der Wahrscheinlichkeit einer Klasse $c$ gegeben die Merkmale $x$ und den Datensatz $D$, wird eine Unabhängigkeitsannahme gemacht. 

Das bedeutet, dass wir "naiv" annehmen, dass die Merkmale $x$ unabhängig voneinander sind, wenn wir die Wahrscheinlichkeit berechnen. In der Realität sind die Merkmale oft nicht unabhängig, aber diese Annahme vereinfacht die Berechnung und macht den Algorithmus effizienter.

Die resultierende Wahrscheinlichkeit $P(c | x,D)$ ist daher eine Schätzung des wahren Wertes. Diese Schätzung kann in einigen Fällen ungenau sein, insbesondere wenn die Unabhängigkeitsannahme stark verletzt ist. Aber in vielen Anwendungsfällen funktioniert der Naive Bayes-Klassifikator trotz dieser Annahme überraschend gut.
**Formal:**
Gegeben sei ein Datensatz $D$ und ein neuer Datenpunkt $x \in Z_1 \times ... \times Z_n$. Der Naive Bayes-Klassifikator $clf_{NaiveBayes_D}$ wird definiert als:

$$clf_{NaiveBayes_D}(x) = \arg\max_{c \in Z} P(c | D)P(x_1 | c,D)P(x_2 | c,D)...P(x_n | c,D)$$

mit

$$P(c | D) = \frac{|{(z, c) \in D}|}{|D|}$$

$$P(x_i | c,D) = \frac{|{(z', c) \in D | z' = (z_1,...,z_n),z_i = x_i}|}{|{(z, c) \in D}|} \text{ für } i = 1,...,n$$

**Beispiel 2** (Fortsetzung für Beispiel 1)

Wir führen Beispiel 1 fort und betrachten wieder den Datenpunkt `x* = (0,1)` (ein in Europa gedrehter Film, der keinen Oskar erhalten hat). Zunächst erhalten wir für die Verteilung der Klassen die Wahrscheinlichkeiten
$$P(c = 0 | D = D_{\text{movies2}}) = \frac{4}{10}$$
$$P(c = 1 | D = D_{\text{movies2}}) = \frac{6}{10}$$
da die Beispiele 2, 4, 6 und 7 als Klasse 0 klassifiziert sind und alle anderen Beispiele als Klasse 1 klassifiziert sind.

In [30]:
bad_films = np.all(D_movies2[['Z']] == 0, axis=1)
good_films = np.all(D_movies2[['Z']] == 1, axis=1)

p_class_0 = np.mean(bad_films)
p_class_1 = np.mean(good_films)

print(f"Wahrscheinlichkeit für Klasse 0: {p_class_0}")
print(f"Wahrscheinlichkeit für Klasse 1: {p_class_1}")

Wahrscheinlichkeit für Klasse 0: 0.4
Wahrscheinlichkeit für Klasse 1: 0.6


Nun schauen wir uns die Wahrscheinlichkeitsverteilungen der einzelnen Merkmalsauspäggungen(`x=0` und `x=1`)  bzgl. der verschiedene 6
Klass(`c=0` und `c=1`) en an. 

Wir erhalten für `Klasse 0` (schlechte Filme):
$$P(x_1 = 0 | c = 0, D = D_{\text{movies2}}) = \frac{3}{4}$$
$$P(x_1 = 1 | c = 0, D = D_{\text{movies2}}) = \frac{1}{4}$$
$$P(x_2 = 0 | c = 0, D = D_{\text{movies2}}) = \frac{3}{4}$$
$$P(x_2 = 1 | c = 0, D = D_{\text{movies2}}) = \frac{1}{4}$$

In [31]:
# Filter the DataFrame for only those rows where Z is 0 (bad films class 0)
bad_films_Z1 = D_movies2[bad_films]['Z1']
bad_films_Z2 = D_movies2[bad_films]['Z2']

# Calculate the probability of Z1 being 0 given that Z is 0
p_Z2_0_given_Z_0 = np.mean(bad_films_Z2 == 0)
p_Z1_0_given_Z_0 = np.mean(bad_films_Z1 == 0)

# Calculate the probability of Z1 being 0 given that Z is 0
p_Z1_0_given_Z_1 = np.mean(good_films_Z1 == 0)
p_Z2_0_given_Z_1 = np.mean(good_films_Z2 == 0)

print(f"Wahrscheinlichkeit für Z1 = 0 gegeben, dass Z = 0: {p_Z1_0_given_Z_0}")
print(f"Wahrscheinlichkeit für Z1 = 1 gegeben, dass Z = 0: {p_Z1_1_given_Z_0}")
print(f"Wahrscheinlichkeit für Z2 = 0 gegeben, dass Z = 0: {p_Z2_0_given_Z_0}")
print(f"Wahrscheinlichkeit für Z2 = 1 gegeben, dass Z = 0: {p_Z2_1_given_Z_0}")

Wahrscheinlichkeit für Z1 = 0 gegeben, dass Z = 0: 0.75
Wahrscheinlichkeit für Z1 = 1 gegeben, dass Z = 0: 0.25
Wahrscheinlichkeit für Z2 = 0 gegeben, dass Z = 0: 0.75
Wahrscheinlichkeit für Z2 = 1 gegeben, dass Z = 0: 0.25


und für `Klasse 1` (gute Filme):
$$P(x_1 = 0 | c = 1, D = D_{\text{movies2}}) = \frac{3}{6}$$
$$P(x_1 = 1 | c = 1, D = D_{\text{movies2}}) = \frac{3}{6}$$
$$P(x_2 = 0 | c = 1, D = D_{\text{movies2}}) = \frac{4}{6}$$
$$P(x_2 = 1 | c = 1, D = D_{\text{movies2}}) = \frac{2}{6}$$

In [32]:
# Filter the DataFrame for only those rows where Z is 1 (good films = class 1)
good_films_Z1 = D_movies2[good_films]['Z1']
good_films_Z2 = D_movies2[good_films]['Z2']

# Calculate the probability of Z2 being 0 given that Z is 0
p_Z1_1_given_Z_0 = np.mean(bad_films_Z1 == 1)
p_Z2_1_given_Z_0 = np.mean(bad_films_Z2 == 1)

# Calculate the probability of Z2 being 0 given that Z is 0
p_Z1_1_given_Z_1 = np.mean(good_films_Z1 == 1)
p_Z2_1_given_Z_1 = np.mean(good_films_Z2 == 1)

print(f"Wahrscheinlichkeit für Z1 = 0 gegeben, dass Z = 1: {p_Z1_0_given_Z_1}")
print(f"Wahrscheinlichkeit für Z1 = 1 gegeben, dass Z = 1: {p_Z1_1_given_Z_1}")
print(f"Wahrscheinlichkeit für Z2 = 0 gegeben, dass Z = 1: {p_Z2_0_given_Z_1}")
print(f"Wahrscheinlichkeit für Z2 = 1 gegeben, dass Z = 1: {p_Z2_1_given_Z_1}")

Wahrscheinlichkeit für Z1 = 0 gegeben, dass Z = 1: 0.5
Wahrscheinlichkeit für Z1 = 1 gegeben, dass Z = 1: 0.5
Wahrscheinlichkeit für Z2 = 0 gegeben, dass Z = 1: 0.6666666666666666
Wahrscheinlichkeit für Z2 = 1 gegeben, dass Z = 1: 0.3333333333333333


Damit gelten für ein Datenpunkt `x* = (0,1)` die folgenden Wahrscheinlichkeiten :

$$
P(c = 0 | D = D_{\text{movies2}})P(x_1 = 0 | c = 0,D = D_{\text{movies2}})P(x_2 = 1 | c = 0,D = D_{\text{movies2}}) = \frac{4}{10} \cdot \frac{3}{4} \cdot \frac{1}{4} = \frac{3}{40}
$$

$$
P(c = 1 | D = D_{\text{movies2}})P(x_1 = 0 | c = 1,D = D_{\text{movies2}})P(x_2 = 1 | c = 1,D = D_{\text{movies2}}) = \frac{6}{10} \cdot \frac{3}{6} \cdot \frac{2}{6} = \frac{1}{10}
$$

In [35]:
def c_given_x_bayes(x):
    # Wahrscheinlichkeit für Klasse 0 (schlechte Filme)
    p_c_0_given_x_star = p_class_0 * (bad_films_Z1 == x[0]).mean() * (bad_films_Z2 == x[1]).mean()
    # Wahrscheinlichkeit für Klasse 1 (gute Filme)
    p_c_1_given_x_star = p_class_1 * (good_films_Z1 == x[0]).mean() * (good_films_Z2 == x[1]).mean()
    
    # Ausgabe der berechneten Wahrscheinlichkeiten
    print(f"Wahrscheinlichkeit für Klasse 0 gegeben x* = {x}: {p_c_0_given_x_star}")
    print(f"Wahrscheinlichkeit für Klasse 1 gegeben x* = {x}: {p_c_1_given_x_star}")
    
# Datenpunkt x* = (0,1)
x_star = np.array([0, 1])
c_given_x_bayes(x_star)


Wahrscheinlichkeit für Klasse 0 gegeben x* = [0 1]: 0.07500000000000001
Wahrscheinlichkeit für Klasse 1 gegeben x* = [0 1]: 0.09999999999999999


Wie man sieht, tritt die Klasse 1 häufiger unter der Merkmalsausprägung $x^*$ auf und deshalb erhalten wir
$$
\text{clf}_{D_{\text{movies2}}}^{\text{Naive Bayes}}(x^*) = 1
$$
Der Film $x^*$ wird also als "gut" klassifiziert.

Betrachten wir nun den neuen Datenpunkt x∗∗ = (1,1) (d. h., ein in Europa gedrehter Film, der einen Oskar gewonnen hat), so erhalten wir:
$$
P(c = 0 | D = D_{\text{movies2}})P(x_1 = 1 | c = 0,D = D_{\text{movies2}})P(x_2 = 1 | c = 0,D = D_{\text{movies2}}) = \frac{4}{10} \cdot \frac{1}{4} \cdot \frac{1}{4} = \frac{1}{40}
$$

$$
P(c = 1 | D = D_{\text{movies2}})P(x_1 = 1 | c = 1,D = D_{\text{movies2}})P(x_2 = 1 | c = 1,D = D_{\text{movies2}}) = \frac{6}{10} \cdot \frac{3}{6} \cdot \frac{2}{6} = \frac{1}{10}
$$

In [36]:
# Datenpunkt x* = (1,1)
c_given_x_bayes(np.array([1, 1]))

Wahrscheinlichkeit für Klasse 0 gegeben x* = [1 1]: 0.025
Wahrscheinlichkeit für Klasse 1 gegeben x* = [1 1]: 0.09999999999999999


und damit ist $$
\text{clf}_{D_{\text{movies2}}}^{\text{Naive Bayes}}(x^{**}) = 1
$$


#### Der Kompakte Code mit `sklearn.naive_bayes`

In [38]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Teilen Sie die Daten in Merkmale X und Klassenlabels y auf
#X = data[:, :2]  # Z1 und Z2 sind die Merkmale
#y = data[:, 2]   # Z ist das Klassenlabel
X = D_movies2.iloc[:, :2].values
y = D_movies2.iloc[:, 2].values

# Erstellen Sie ein Naive Bayes Modell
clf = GaussianNB().fit(X, y)

# Machen Sie Vorhersagen mit den Daten
y_pred = clf.predict(X)
print(y_pred)

# Berechnen Sie die Genauigkeit des Modells
accuracy = accuracy_score(y, y_pred)

print(f"Modellgenauigkeit: {accuracy}")


[1 1 0 1 1 0 0 1 1 1]
Modellgenauigkeit: 0.7


In [41]:
def c_given_x_compact_bayes(clf, x):
    # Vorhersage für x*
    y_pred_x = clf.predict(x)
    print(y_pred_x)
    print(f"Vorhersage für x* = {x}: {y_pred_x[0]}")
    
# Der neue Datenpunkt x*
x_star = np.array([[0, 1]])
x_double_star = np.array([[1, 1]])

c_given_x_compact_bayes(clf, x_star)
c_given_x_compact_bayes(clf, x_double_star)

[1]
Vorhersage für x* = [[0 1]]: 1
[1]
Vorhersage für x* = [[1 1]]: 1


## 2.5.4. Kontinuierliche Merkmale <a name="2_5_4"></a>
---
Der Naive Bayes-Klassifikator kann auch auf kontinuierliche Merkmale angewendet werden, indem wir von einer Wahrscheinlichkeitsverteilung zu einer Wahrscheinlichkeitsdichte wechseln. Anstatt die Verteilung $$P(x_i | c,D)$$ direkt aus den Daten zu berechnen, verwenden wir eine abgeschätzte Dichte $$p(x_i | c,D)$$. Die Herausforderung besteht darin, diese Dichte geeignet abzuschätzen, da zusätzliche Annahmen erforderlich sind.

Betrachten Sie den Datensatz `D_kont`. Hier haben wir ein kontinuierliches Merkmal `Z1` mit Merkmalsraum $$\mathbb{R}$$. Eine genaue Inspektion der Verteilung der Merkmalsausprägungen legt nahe, dass `x1` für die Klasse 0 normalverteilt mit Erwartungswert 2 und für die Klasse 1 normalverteilt mit Erwartungswert 4 is
| 10 | 3.8 | 1 |

In [52]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB

# Datensatz D_kont als DataFrame
D_kont = pd.DataFrame({
    'Z1': [4.0, 2.2, 3.9, 4.0, 1.9, 4.1, 2.0, 1.9, 2.0, 3.8],
    'Z': [1, 0, 1, 1, 0, 1, 0, 0, 0, 1]
})

# Aufteilen des Datensatzes in Merkmale und Zielvariable
X = D_kont[['Z1']]   # Merkmale (Z1)
y = D_kont['Z']  # Zielvariable (Z)

# Erstellen des Naiven Bayes-Klassifikators
clf = GaussianNB().fit(X, y)

# Vorhersage für einen gegebenen Datenpunkt x*
x_star = np.array([[2.5]])  # z.B. x* = 2.5
c_given_x_compact_bayes(clf, x_star)


[0]
Vorhersage für x* = [[2.5]]: 0


C:\Users\maayo\PycharmProjects\Sem2_ML\venv\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


## 2.5.5.Zusammenfassung <a name="2_5_5"></a>


Der folgende Code verwendet den Naiven Bayes-Klassifikator, um einen synthetischen Klassifikationsdatensatz zu modellieren.

In [4]:
# Diese Zeile erzeugt einen zufälligen Klassifikationsdatensatz mit der Funktion `make_classification` aus der `sklearn.datasets` Bibliothek. `X` enthält die Merkmale und `y` enthält die Klassenlabels.
X, y = make_classification()

print(X)
print(y)


[[-1.2438174   2.64178576 -2.44433124 ...  2.35845448  0.02988481
   0.02296293]
 [-0.42104111 -0.75854682 -0.02426976 ... -1.20398791 -2.30845398
   0.55613795]
 [-0.5688051  -0.73101635  0.40454672 ... -0.8498266  -0.96568188
   0.7168087 ]
 ...
 [ 0.49135006  1.5432398  -0.71724746 ...  1.89329607 -2.20603017
  -1.19670352]
 [-1.32489877 -0.98367453  0.17375312 ... -1.41242742 -0.67473117
  -1.03197684]
 [ 0.74248278 -0.13160448  1.28153067 ...  0.72390933  3.48271338
  -0.13537485]]
[0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1
 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0
 1 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1]


In [8]:
#Hier wird ein Naiver Bayes-Klassifikator mit einer Gaußschen Verteilungsannahme erstellt und auf den Datensatz angewendet.
clf = GaussianNB().fit(X,y)

y_pred = clf.predict(X)
y_pred

array([1, 1, 0, 1, 1, 0, 0, 1, 1, 1])

In [6]:

metrics.accuracy_score(y,clf.predict(X))

0.99